# Anthropic Tool calling capabilities with Unity Catalog

## Prerequisites

**API Key**
To run this tutorial, you will need an Anthropic API key. For testing purposes, you can generate a new account, and use your evaluation test key (no credit card required!).

Once you have acquired your key, set it to the environment variable `ANTHROPIC_API_KEY`.

Below, we validate that this key is set properly in your environment.

**Packages**

To interface with both UnityCatalog and Anthropic, you will need to install the following packages:

```shell
pip install anthropic unitycatalog-anthropic
```

In [1]:
import os

assert "ANTHROPIC_API_KEY" in os.environ, (
    "Please set the ANTHROPIC_API_KEY environment variable to your Anthropic API key"
)

## Configuration and Client setup

In order to connect to your Unity Catalog server, you'll need an instance of the `ApiClient` from the `unitycatalog-client` package. 

> Note: If you don't already have a Catalog and a Schema created, be sure to create them before running this notebook and adjust the `CATALOG` and `SCHEMA` variables below to suit.

In [ ]:
from unitycatalog.ai.anthropic.toolkit import UCFunctionToolkit
from unitycatalog.ai.anthropic.utils import generate_tool_call_messages
from unitycatalog.ai.core.client import UnitycatalogFunctionClient
from unitycatalog.client import ApiClient, Configuration

In [3]:
config = Configuration()
config.host = "http://localhost:8080/api/2.1/unity-catalog"

# The base ApiClient is async
api_client = ApiClient(configuration=config)

client = UnitycatalogFunctionClient(api_client=api_client)

CATALOG = "AICatalog"
SCHEMA = "AISchema"

## Define a function and register it to Unity Catalog

In this next section, we'll be defining a placeholder Python function and creating it within Unity Catalog so that it can be retrieved and used as a tool by Anthropic's Claude model. 

There are a few things to keep in mind when creating functions for use with the `create_python_function` API:

- Ensure that your have properly defined types for all arguments and for the return of the function.
- Ensure that you have a Google-style docstring defined that includes descriptions for the function, each argument, and the return of the function. This is critical, as these are used to populate the metadata associated with the function within Unity Catalog, providing contextual data for an LLM to understand when and how to call the tool associated with this function.
- If there are packages being called that are not part of core Python, ensure that the import statements are locally scoped (defined within the function body).

In [4]:
def fetch_weather(location: str) -> str:
    """
    Fetches the current weather in celsius for a given location.

    Args:
        location (str): The location to fetch the weather for.

    Returns:
        str: The current weather in celsius for the given location.
    """

    return "88.2 F"

In [5]:
client.create_python_function(func=fetch_weather, catalog=CATALOG, schema=SCHEMA, replace=True)

FunctionInfo(name='fetch_weather', catalog_name='AICatalog', schema_name='AISchema', input_params=FunctionParameterInfos(parameters=[FunctionParameterInfo(name='location', type_text='STRING', type_json='{"name": "location", "type": "string", "nullable": false, "metadata": {"comment": "The location to fetch the weather for."}}', type_name=<ColumnTypeName.STRING: 'STRING'>, type_precision=None, type_scale=None, type_interval_type=None, position=0, parameter_mode=None, parameter_type=None, parameter_default=None, comment='The location to fetch the weather for.')]), data_type=<ColumnTypeName.STRING: 'STRING'>, full_data_type='STRING', return_params=None, routine_body='EXTERNAL', routine_definition='return "88.2 F"', routine_dependencies=None, parameter_style='S', is_deterministic=True, sql_data_access='NO_SQL', is_null_call=False, security_type='DEFINER', specific_name='fetch_weather', comment='Fetches the current weather in celsius for a given location.', properties='null', full_name='AIC

## Create a Toolkit instance of the function(s)

Now that the function has been created within Unity Catalog, we can use the `unitycatalog-anthropic` package to create a toolkit instance that Anthropic will 'understand' as a valid tool in its APIs. 

In [6]:
toolkit = UCFunctionToolkit(function_names=[f"{CATALOG}.{SCHEMA}.fetch_weather"], client=client)

tools = toolkit.tools

In [7]:
tools

[AnthropicTool(name='AICatalog__AISchema__fetch_weather', description='Fetches the current weather in celsius for a given location.', input_schema={'type': 'object', 'properties': {'location': {'default': None, 'description': 'The location to fetch the weather for.', 'title': 'Location', 'type': 'string'}}, 'required': []})]

## Call Claude with tools

With our toolkit instance ready, we can now pass in our tools to a call to Anthropic, giving Claude the ability to request for 'answers' from tools that have been provided with the request.

In [8]:
# Interface with Anthropic via their SDK
import anthropic

anthropic_client = anthropic.Anthropic()

multi = "What's the weather in Nome, AK and in Death Valley, CA?"

question = [{"role": "user", "content": multi}]

response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=question,
)

response

Message(id='msg_012UX8sMYq55q1mR3HzbaWDL', content=[TextBlock(text="To answer your question about the weather in Nome, AK and Death Valley, CA, I'll need to use the weather fetching tool twice, once for each location. Let me do that for you now.", type='text'), ToolUseBlock(id='toolu_01MRhL9hHEcYsSBoM1hXXeYb', input={'location': 'Nome, AK'}, name='AICatalog__AISchema__fetch_weather', type='tool_use'), ToolUseBlock(id='toolu_01NwcjWquf1bJvzjwxyi43H2', input={'location': 'Death Valley, CA'}, name='AICatalog__AISchema__fetch_weather', type='tool_use')], model='claude-3-5-sonnet-20240620', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(input_tokens=406, output_tokens=155))

## Use the helper utility to call the tools and continue the conversation

With the response from Claude being a request to execute specific tools with parameters, we can utilize the `generate_tool_call_messages` helper utility from `unitycatalog.ai.anthropic.utils`. This function will process the response from Anthropic, parse the data, determine whether tool(s) need to be called, call the tool(s), and formulate the entire response structure for submitting the follow-on response call back to Anthropic.

> Note: If not using this helper functionality, keep in mind that Anthropic will submit multiple tool request calls if it is deemed as necessary. If complex multi-turn tool calls are needed (i.e. a response from `Tool A` is needed to call `Tool B`, additional `ToolUseBlock` responses will be generated on a subsequent call), you will need to handle this yourself. The utility function, on the other hand, is capable of handling this logic and will continue to process tool calls until the response structure contains no such elements.

In [9]:
# Call the Unity Catalog function and construct the required formatted response history for a subsequent call to Anthropic
tool_messages = generate_tool_call_messages(
    response=response, client=client, conversation_history=question
)

# Call the Anthropic client with the parsed tool response from executing the Unity Catalog function
tool_response = anthropic_client.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=1024,
    tools=tools,
    messages=tool_messages,
)

tool_response

Message(id='msg_01Vjv77qn7LxVu4EwozDSjQ6', content=[TextBlock(text="Thank you for waiting. I've fetched the current weather information for both locations. Here's what I found:\n\n1. Nome, AK: The current temperature is 88.2°F (31.2°C).\n2. Death Valley, CA: The current temperature is also 88.2°F (31.2°C).\n\nInterestingly, both locations are showing the same temperature right now, which is quite unusual given their very different geographical locations and typical climate patterns. Nome, Alaska is typically much cooler than Death Valley, California, especially during most of the year.\n\nDeath Valley is known for its extremely hot temperatures, especially during summer months, so this temperature is not unusual for that location. However, for Nome, Alaska, this is an exceptionally high temperature, as it's located near the Arctic Circle and generally experiences much cooler weather.\n\nIf you'd like more detailed weather information or have any questions about this unusual weather pat